In [1]:
# Cell 1: Installation
!pip install owlready2


[notice] A new release of pip is available: 24.3.1 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from owlready2 import *
import json
import re
from pathlib import Path

In [3]:

# We define namespaces
dul_onto = get_ontology("http://www.ease-crc.org/ont/DUL.owl").load()
dul_ns = get_namespace("http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#")
onto = get_ontology("http://www.semanticweb.org/gamematcher.owl")
onto.imported_ontologies.append(dul_onto)

with onto:
    onto.metadata.comment.append("Andreea Scrob, Edoardo Tommasi")
    onto.metadata.comment.append("GameMatcher is an intelligent companion system that assists users in finding video games compatible with their computer.")
    onto.metadata.label.append("GameMatcher Ontology")

    # TAXONOMY (CLASSES)
    #  USER
    class User(Thing):
        comment = ["An Agent who interacts with the system to find compatible video games."]
        label = ["User"]
        is_a = [dul_ns.Agent]

    # COMPUTER and COMPONENTS
    # Note: A Computer is a Physical Object, not a subclass of Component.
    class Computer(Thing):
        comment = ["A Physical Object consisting of hardware components capable of executing software."]
        label = ["Computer"]
        is_a = [dul_ns.PhysicalObject]

    class Component(Thing):
        comment = ["A Physical Object that forms a constituent part of a computing system."]
        label = ["Component"]
        is_a = [dul_ns.PhysicalObject]

    # Specific Hardware Classes
    class CPU(Component):
        comment = ["A Hardware Component that acts as the primary processing unit."]
        label = ["CPU"]

    class GPU(Component):
        comment = ["A Hardware Component specialized for rendering graphics."]
        label = ["GPU"]

    class RAM(Component):
        comment = ["A Hardware Component used for temporary data storage."]
        label = ["RAM"]

    class Storage(Component):#da specificare nella relazione che questo diventa Storage poi lo spazio disponibile viene preso con la dataproperty
        comment = ["A Hardware Component used for data retention (e.g., HDD or SSD)."]
        label = ["Storage"]

    # Specific Software Classes
    class OperatingSystem(Thing):
        comment = ["A Software Component that manages computer hardware and resources."]
        label = ["Operating System"]
        is_a = [dul_ns.InformationObject]

    #  VIDEOGAME and REQUIREMENTS
    class VideoGame(Thing):
        comment = ["An Information Object representing a digital game available for play."]
        label = ["Video Game"]
        is_a = [dul_ns.InformationObject]

    class MinimumRequirement(Thing):
        comment = ["A Requirement specifying the absolute minimum specifications for a game to run."]
        label = ["Minimum Requirement"]
        is_a = [dul_ns.Description]


    #  ATTRIBUTES and CONCEPTS
    #todo: chiedere al prof se possiamo aggiungere GameAttribute
    class GameAttribute(Thing):
        comment = ["A Concept representing a characteristic feature of a video game."]
        label = ["Game Attribute"]
        is_a = [dul_ns.Concept]

    class GameGenre(GameAttribute):
        comment = ["A classification based on gameplay interaction (e.g., RPG, Strategy)."]
        label = ["Game Genre"]

    class PlayerMode(GameAttribute):
        comment = ["Defines the number of players supported (e.g., Single-player)."]
        label = ["Player Mode"]

    class PEGIRating(GameAttribute):
        comment = ["Indicates the suitability of content for different age groups."]
        label = ["PEGI Rating"]

    # 1. Top-Level Disjointness
    # A User is not a Computer, which is not a Game, etc.
    AllDisjoint([User, Computer, Component, OperatingSystem, VideoGame, MinimumRequirement, GameAttribute])

    # 2. Hardware Component Disjointness
    # A CPU is not a GPU, RAM, or Storage.
    AllDisjoint([CPU, GPU, RAM, Storage])

    # 3. Attribute Disjointness
    # A Genre is not a PEGI Rating or a Player Mode.
    AllDisjoint([GameGenre, PlayerMode, PEGIRating])


    # OBJECT PROPERTIES (RELATIONS)
    # Relations: Computer -> Components
    class hasComponent(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]
        range = [Component]
        comment = ["Generic relation linking a computer to its parts."]
        label = ["has component"]


    # Sub-properties for specificity
    class hasCPU(hasComponent, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]; range = [CPU]
        comment = ["Relation linking components to the computer's cpu."]
        label = ["has CPU"]

    class hasGPU(hasComponent, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]; range = [GPU]
        comment = ["Relation linking components to the computer's gpu."]
        label = ["has GPU"]

    class hasRAM(hasComponent, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]; range = [RAM]
        comment = ["Relation linking components to the computer's ram."]
        label = ["has RAM"]

    class hasStorage(hasComponent, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]; range = [Storage]
        comment = ["Relation linking components to the computer's  available storage."]
        label = ["has storage"]

    class hasOS(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]; range = [OperatingSystem]
        comment = ["Relation linking components to the computer's os."]
        label = ["has operating system"]


    # Relations: Videogame -> Requirements
    class hasMinRequirement(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [VideoGame]
        range = [MinimumRequirement]
        comment = ["Relation linking videogame to its component requirements to make it run."]
        label = ["has minimum requirement"]


    # Relations: User -> Computer
    class hasComputer(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [User]
        range = [Computer]
        comment = ["Relation linking user to the computer he owns."]
        label = ["has computer device"]



    # Relations: Attribute
    class hasGenre(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [VideoGame]; range = [GameGenre]
        comment = ["Relation linking videogame to its genre."]
        label = ["has genre"]


    class preferredGenre(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [User]; range = [GameGenre]
        comment = ["Relation linking user to its preferred game's genre."]
        label = ["prefers genre"]

    class hasPlayerMode(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [VideoGame]; range = [PlayerMode]
        comment = ["Relation linking videogame to its player mode."]
        label = ["has player mode"]


    class preferredPlayerMode(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [User]; range = [PlayerMode]
        comment = ["Relation linking user to its preferred game mode."]
        label = ["prefers player mode"]


    class hasPEGI(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [VideoGame]; range = [PEGIRating]
        comment = ["Relation linking videogame to its age rating."]
        label = ["has PEGI rating"]


    # Derived Relation (for Compatibility Checking)
    class isCompatibleWith(ObjectProperty, SymmetricProperty, IrreflexiveProperty):
        domain = [VideoGame]
        range = [Computer]
        comment = ["Inferred relation: true if the computer meets all game requirements."]
        label = ["is compatible with"]



    # DATA PROPERTIES (ACTIONABLE LOGIC)

    # BENCHMARK SCORE (CPU and GPU)
    # Game Requirement Property
    class requiresCPUBenchmarkScore(DataProperty, FunctionalProperty):
        domain = [MinimumRequirement]
        range = [int]
        label = ["requires CPU benchmark score"]

    class requiresGPUBenchmarkScore(DataProperty, FunctionalProperty):
        domain = [MinimumRequirement]
        range = [int]
        label = ["requires GPU benchmark score"]


    # Hardware Component Property (NEW)
    class hasBenchmarkScore(DataProperty, FunctionalProperty):
        domain = [CPU | GPU] #TODO: da capire quanto e pericoloso questo or perche se metti la virgola [CPU, GPU] da errore per il dijoint 
        range = [int]
        label = ["has benchmark score"]


    #  MEMORY (RAM)
    class hasMemorySizeGB(DataProperty, FunctionalProperty):
        domain = [RAM]
        range = [float]
        comment = ["The amount of  RAM memory in Gigabytes."]
        label = ["has memory size (GB)"]


    class requiresMemoryGB(DataProperty, FunctionalProperty):
        domain = [MinimumRequirement]
        range = [float]
        comment = ["Minimum RAM required in Gigabytes."]
        label = ["requires memory (GB)"]


    #  STORAGE
    class hasStorageSizeGB(DataProperty, FunctionalProperty):
        domain = [Storage]
        range = [float]
        comment = ["Total available disk space in Gigabytes."]
        label = ["has storage size (GB)"]


    class requiresStorageSpaceGB(DataProperty, FunctionalProperty):
        domain = [MinimumRequirement]
        range = [float]
        comment = ["Installation size required in Gigabytes."]
        label = ["requires storage space (GB)"]


    # OS LOGIC
    class hasOSVersionValue(DataProperty, FunctionalProperty):
        domain = [OperatingSystem]
        range = [float]
        comment = ["Numeric version of the OS (e.g., 10.0, 11.0)."]
        label = ["has OS version value"]


    class requiresMinOSVersionValue(DataProperty, FunctionalProperty):
        domain = [MinimumRequirement]
        range = [float]
        comment = ["Minimum numeric OS version required."]
        label = ["requires min OS version"]



   #  BUDGET and PRICING
    class hasPrice(DataProperty, FunctionalProperty):
        domain = [VideoGame]
        range = [float]
        comment = ["The monetary cost required to purchase the digital game."]
        label = ["has price"]


    class hasBudget(DataProperty, FunctionalProperty):
        domain = [User]
        range = [float]
        comment = ["The maximum monetary limit the user is willing to spend."]
        label = ["has budget"]


    #  AGE and PEGI
    class hasAge(DataProperty, FunctionalProperty):
        domain = [User]
        range = [int]
        comment = ["The chronological age of the user (in years), used for PEGI checks."]
        label = ["has age"]


    class hasPEGIAgeThreshold(DataProperty, FunctionalProperty):
        domain = [PEGIRating]
        range = [int]
        comment = ["The numeric age threshold (e.g., 18)."]
        label = ["has PEGI threshold"]




In [4]:

# UTILITY FUNCTIONS (Parsing & Matching)
def clean_iri(text):
    """
    Removes illegal characters to create a valid IRI/URI.
    Example: "Windows 10" -> "Windows_10"
    """
    if not text: return "Unknown"
    # Replace spaces and non-alphanumeric chars with underscore
    clean = re.sub(r'[^a-zA-Z0-9]', '_', str(text))
    # Remove multiple consecutive underscores
    clean = re.sub(r'_+', '_', clean)
    return clean.strip('_')

def parse_gb(text):
    """
    Extracts Gigabytes (float) from strings like '8 GB RAM' or '100 GB available'.
    Used for: hasMemorySizeGB, requiresMemoryGB, hasStorageSizeGB.
    """
    if not text: return 0.0
    # Finds the first number in the string
    match = re.search(r'(\d+)', str(text))
    if match:
        return float(match.group(1))
    return 0.0

def parse_price(text):
    """
    Extracts price from strings like '39,99€' or handles 'Free To Play'.
    Used for: hasPrice.
    """
    if not text: return 0.0
    if "Free" in str(text): return 0.0
    
    # Replace comma with dot for float conversion
    text = str(text).replace(',', '.')
    # Extract the number (allows decimals)
    match = re.search(r'(\d+\.\d+)', text)
    if match:
        return float(match.group(1))
    return 0.0

def parse_os_version(os_string):
    """
    Extracts the OS version number from strings like 'Windows 10 64-bit'.
    Used for: hasOSVersionValue, requiresMinOSVersionValue.
    """
    if not os_string or os_string == "N/A":
        return None
    
    # Look for numbers like 10, 11, or 7.
    # Note: This is a simple parser. For 'Windows 10', it returns 10.0.
    match = re.search(r'(\d+)', str(os_string))
    if match:
        return float(match.group(1))
    return None

def find_hardware_score(req_text, hardware_db):
    """
    Searches for the benchmark score in the hardware database.
    
    Args:
        req_text: String from Steam (e.g., 'NVIDIA GeForce GTX 1060')
        hardware_db: List of dictionaries from the benchmark JSON
        
    Returns:
        int: The benchmark score (Mark) or 0 if not found.
    """
    if not req_text: return 0  # Return 0 instead of None for easier numerical comparison
    
    # Normalize the requirement text
    req_clean = str(req_text).lower().replace('-', ' ').replace('_', ' ')
    
    best_match = None
    best_score = 0
    
    # Logic: iterate through the benchmark DB
    for item in hardware_db:
        # Normalize benchmark name
        bench_name = item['name'].lower().replace('-', ' ')
        
        # Check if the benchmark name is inside the requirement string
        # Example: "geforce gtx 1060" in "nvidia geforce gtx 1060" -> TRUE
        if bench_name in req_clean:
            # We look for the "longest" match to be more specific
            # (e.g., matching "GTX 1060 Ti" is better than just "GTX 1060")
            if best_match is None or len(bench_name) > len(best_match):
                best_match = bench_name
                best_score = int(item['mark'])
    
    return best_score

In [5]:
# Set the correct paths for your environment
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / "Scraping" / "cleaning"

try:
    with open(DATA_DIR / "cpu_filtered.json", 'r', encoding='utf-8') as f: cpu_data = json.load(f)
    with open(DATA_DIR / "gpu_filtered.json", 'r') as f: gpu_data = json.load(f)
    with open(DATA_DIR / "steam_parsed_CLEAN.json", 'r') as f: game_data = json.load(f)
    print("JSON files loaded successfully.")
except FileNotFoundError:
    print(f"Error: Files not found in {DATA_DIR}")
    cpu_data, gpu_data, game_data = [], [], []

# --- ONTOLOGY POPULATION ---
with onto:

    # --- HELPER FUNCTION FOR STANDARD MEMORY CREATION ---
    def create_standard_memories():
        """
        Creates standard individuals for RAM (including high-end for users).
        Useful so users can link to existing RAM entities.
        """
        
        # Extended list with high-end sizes for users (32, 64, 128 GB)
        standard_ram_sizes = [2, 4, 6, 8, 12, 16, 24, 32, 64, 128]
        
        print(f"Creating Standard RAM: {standard_ram_sizes} GB")

        for size in standard_ram_sizes:
            iri = f"RAM_{size}GB"
            # Search if exists, otherwise create
            if not onto.search_one(iri=f"*{iri}"):
                r = onto.RAM(iri)
                r.label = [f"{size} GB RAM"]
                r.hasMemorySizeGB = float(size)

    # --- POPULATION EXECUTION ---
    
    # 1. Base Hardware (CPU/GPU from benchmark)
    print("Populating CPU & GPU...")
    cpu_db = cpu_data 
    gpu_db = gpu_data
    
    for cpu in cpu_data:
        cpu_iri = clean_iri(cpu['name'])
        if not onto.search_one(iri=f"*CPU_{cpu_iri}"):
            new_cpu = onto.CPU(f"CPU_{cpu_iri}")
            new_cpu.label = [cpu['name']]
            new_cpu.hasBenchmarkScore = int(cpu['mark'])
        
    for gpu in gpu_data:
        gpu_iri = clean_iri(gpu['name'])
        if not onto.search_one(iri=f"*GPU_{gpu_iri}"):
            new_gpu = onto.GPU(f"GPU_{gpu_iri}")
            new_gpu.label = [gpu['name']]
            new_gpu.hasBenchmarkScore = int(gpu['mark'])

    # 2. Creating Standard Individuals (OS and RAM)
    # These are created so User PCs can link to them.
    print("Creating Standard Individuals...")
    
    # Standard OS
    for ver in [7, 8, 10, 11]:
        ver_float = float(ver)
        os_iri = f"OS_Windows_{ver}"
        if not onto.search_one(iri=f"*{os_iri}"):
            new_os = onto.OperatingSystem(os_iri)
            new_os.label = [f"Windows {ver}"]
            new_os.hasOSVersionValue = ver_float

    # Standard RAM (Create them in the graph)
    create_standard_memories()

    # 3. Game Population
    print("Populating Games...")
    
    for game in game_data:
        game_name = game.get('NAME', 'UnknownGame')
        game_iri = clean_iri(game_name)
        new_game = onto.VideoGame(f"Game_{game_iri}")
        new_game.label = [game_name]
        
        # Price & PEGI
        new_game.hasPrice = parse_price(game.get('PRICE'))
        pegi_text = game.get('PEGI', 'N/A')
        if pegi_text and pegi_text != 'N/A' and not pegi_text.startswith('#'):
            pegi_iri = clean_iri(pegi_text)
            p = onto.PEGIRating(f"PEGI_{pegi_iri}")
            p.label = [pegi_text]
            age_match = re.search(r'\d+', pegi_text)
            if age_match: p.hasPEGIAgeThreshold = int(age_match.group(0))
            new_game.hasPEGI = p

        # Modes
        mode_text = game.get('MODE', '')
        if mode_text:
            for m in [x.strip() for x in mode_text.split(',')]:
                m_iri = clean_iri(m)
                mod = onto.PlayerMode(f"Mode_{m_iri}")
                mod.label = [m]
                new_game.hasPlayerMode.append(mod)
        
        # Genres
        genre_text = game.get('GENRE', '')
        if genre_text:
            for g in [x.strip() for x in genre_text.split(',')]:
                g_iri = clean_iri(g)
                gen = onto.GameGenre(f"Genre_{g_iri}")
                gen.label = [g]
                new_game.hasGenre.append(gen)

        # Requirements
        req_iri = f"Req_{game_iri}"
        new_req = onto.MinimumRequirement(req_iri)
        new_game.hasMinRequirement = new_req
        new_req.label = [f"Minimum Requirements for {game_name}"]

        # OS Requirement (Linking to the Objects we created earlier)
        os_text = game.get('OS', '')
        
        # 1. Extract the number so we can match your standard individuals
        # (Assuming parse_os_version returns a float like 10.0, 7.0, etc.)
        version_num = parse_os_version(os_text) 

        if version_num:
            new_req.requiresMinOSVersionValue = version_num

        # RAM Requirement (Data Property Only)
        ram_text = game.get('RAM', '')
        ram_val = parse_gb(ram_text)
        if ram_val > 0:
            new_req.requiresMemoryGB = ram_val

        # ROM Requirement

        rom_text = game.get('STORAGE', '')
        rom_val = parse_gb(rom_text)
        if rom_val > 0:
            new_req.requiresStorageSpaceGB = rom_val

        # CPU/GPU Scores (Logic using find_hardware_score)
        # GPU
        valid_gpus = []
        for k in ['GPU_0', 'GPU_1', 'GPU_2']:
            val = game.get(k)
            score = find_hardware_score(val, gpu_db)
            if score and score > 0: valid_gpus.append((int(score), val))
        if valid_gpus:
            valid_gpus.sort(key=lambda x: x[0])
            new_req.requiresGPUBenchmarkScore = valid_gpus[0][0]

        # CPU
        valid_cpus = []
        for k in ['CPU_0', 'CPU_1', 'CPU_2']:
            val = game.get(k)
            score = find_hardware_score(val, cpu_db)
            if score and score > 0: valid_cpus.append((int(score), val))
        if valid_cpus:
            valid_cpus.sort(key=lambda x: x[0])
            new_req.requiresCPUBenchmarkScore = valid_cpus[0][0]

print("Population completed. Standard individuals and Game Requirements created.")

JSON files loaded successfully.
Populating CPU & GPU...
Creating Standard Individuals...
Creating Standard RAM: [2, 4, 6, 8, 12, 16, 24, 32, 64, 128] GB
Populating Games...
Population completed. Standard individuals and Game Requirements created.


In [ ]:

# USER CREATION LOGIC (User ABox)
def create_user_with_existing_hardware(user_name, age, budget, pc_specs, preferences):
    """
    Creates a User and their Computer instance. 
    It searches for existing CPU, GPU, RAM, and OS individuals in the Knowledge Graph 
    to link them instead of creating duplicates.
    """
    with onto:
        # 1. USER & COMPUTER CREATION
        safe_user_name = clean_iri(user_name)
        
        # Create User
        user = onto.User(f"User_{safe_user_name}")
        user.label = [user_name]
        user.hasAge = age
        user.hasBudget = float(budget)
        
        # Create Computer (Unique to this user)
        pc = onto.Computer(f"PC_{safe_user_name}")
        pc.label = [f"{user_name}'s Computer"]
        user.hasComputer.append(pc)
        
        # 2. HARDWARE LINKING
        
        # --- CPU LINKING ---
        if "cpu_name" in pc_specs:
            target_cpu_iri = f"CPU_{clean_iri(pc_specs['cpu_name'])}"
            existing_cpu = onto.search_one(iri=f"*{target_cpu_iri}")
            
            if existing_cpu:
                print(f"[MATCH] CPU found: {existing_cpu.name}")
                pc.hasCPU = existing_cpu
            else:
                print(f"[WARN] CPU '{pc_specs['cpu_name']}' not found. Creating local instance.")
                # Fallback: Create local CPU with manual score (if provided)
                new_cpu = onto.CPU(f"CPU_{safe_user_name}_Local")
                new_cpu.label = [pc_specs['cpu_name']]
                new_cpu.hasBenchmarkScore = pc_specs.get('manual_score', 0)
                pc.hasCPU = new_cpu

        # --- GPU LINKING ---
        if "gpu_name" in pc_specs:
            target_gpu_iri = f"GPU_{clean_iri(pc_specs['gpu_name'])}"
            existing_gpu = onto.search_one(iri=f"*{target_gpu_iri}")
            
            if existing_gpu:
                print(f"[MATCH] GPU found: {existing_gpu.name}")
                pc.hasGPU = existing_gpu
            else:
                # Se non esiste nel DB, ne creiamo una locale basata solo sul nome e (opzionalmente) score manuale
                print(f"[WARN] GPU '{pc_specs['gpu_name']}' not found. Creating local instance.")
                new_gpu = onto.GPU(f"GPU_{safe_user_name}_Local")
                new_gpu.label = [pc_specs['gpu_name']]
                new_gpu.hasBenchmarkScore = pc_specs.get('manual_score', 0)
                pc.hasGPU = new_gpu

        # --- OS LINKING ---
        if "os_ver" in pc_specs:
            os_val = float(pc_specs['os_ver'])
            # Search for standard OS created in population step (e.g., OS_Windows_10)
            target_os_iri = f"OS_Windows_{int(os_val)}"
            existing_os = onto.search_one(iri=f"*{target_os_iri}")
            
            if existing_os:
                print(f"[MATCH] OS found: {existing_os.name}")
                pc.hasOS = existing_os
            else:
                # Fallback for non-standard versions
                print(f"[WARN] OS 'Windows {os_val}' not found. Creating local instance.")
                new_os = onto.OperatingSystem(f"OS_{safe_user_name}_Local")
                new_os.label = [f"Windows {pc_specs['os_ver']}"]
                new_os.hasOSVersionValue = os_val
                pc.hasOS = new_os

        # --- RAM LINKING (Standardized) ---
        if "ram_gb" in pc_specs:
            ram_amount = float(pc_specs['ram_gb'])
            # Construct standard IRI (e.g., RAM_16GB, RAM_64GB)
            target_ram_iri = f"RAM_{int(ram_amount)}GB"
            existing_ram = onto.search_one(iri=f"*{target_ram_iri}")
            
            if existing_ram:
                print(f"[MATCH] Standard RAM found: {existing_ram.name}")
                pc.hasRAM.append(existing_ram)
            else:
                # Create as STANDARD individual so future users can link to it
                print(f"[INFO] Standard RAM '{int(ram_amount)}GB' not found. Creating new STANDARD individual.")
                new_standard_ram = onto.RAM(target_ram_iri) 
                new_standard_ram.label = [f"{int(ram_amount)} GB RAM"]
                new_standard_ram.hasMemorySizeGB = ram_amount
                pc.hasRAM.append(new_standard_ram)
            
        # --- STORAGE (Physical Component) ---
        if "storage_gb" in pc_specs:
            storage = onto.Storage(f"Storage_{safe_user_name}")
            storage.label = [f"{pc_specs['storage_gb']} GB Drive"]
            storage.hasStorageSizeGB = float(pc_specs['storage_gb'])
            pc.hasStorage.append(storage)
            
        # 3. PREFERENCES (Genres & Modes)
        for g_name in preferences.get('genres', []):
            g_iri = clean_iri(g_name)
            genre = onto.search_one(iri=f"*Genre_{g_iri}") or onto.search_one(iri=f"*{g_iri}")
            
            if genre and isinstance(genre, onto.GameGenre):
                user.preferredGenre.append(genre)
            else:
                new_genre = onto.GameGenre(f"Genre_{g_iri}")
                new_genre.label = [g_name]
                user.preferredGenre.append(new_genre)

        for m_name in preferences.get('modes', []):
            m_iri = clean_iri(m_name)
            mode = onto.search_one(iri=f"*Mode_{m_iri}") or onto.search_one(iri=f"*{m_iri}")
            
            if mode and isinstance(mode, onto.PlayerMode):
                user.preferredPlayerMode.append(mode)
            else:
                new_mode = onto.PlayerMode(f"Mode_{m_iri}")
                new_mode.label = [m_name]
                user.preferredPlayerMode.append(new_mode)
    
    return user


# EXECUTION: CREATING TEST USERS

print("\n--- Creating Test Users ---")

# 1. Mark: High-End PC
user_mark = create_user_with_existing_hardware(
    user_name="Mark",
    age=25,
    budget=80.0,
    pc_specs={
        "cpu_name": "Intel Core i7-4790K", 
        "gpu_name": "NVIDIA GeForce GTX 1050",     
        "ram_gb": 64.0,  
        "storage_gb": 1000.0,
        "os_ver": 11.0    
    },
    preferences={
        "genres": ["Action", "RPG"],
        "modes": ["Multiplayer"]
    }
)

# 2. Giulio: Budget Gamer
user_giulio = create_user_with_existing_hardware(
    user_name="Giulio",
    age=16, 
    budget=20.0,
    pc_specs={
        "cpu_name": "Intel Core i5-6600K", 
        "gpu_name": "NVIDIA GeForce GTX 1060",
        "ram_gb": 8.0,    
        "storage_gb": 500.0,
        "os_ver": 10.0
    },
    preferences={
        "genres": ["Simulation", "Indie"],
        "modes": ["Single-player"]
    }
)

# 3. Dev_User: Workstation (Ultra High Specs)
user_workstation = create_user_with_existing_hardware(
    user_name="Dev_User",
    age=30,
    budget=500.0,
    pc_specs={
        "cpu_name": "AMD Ryzen 9 7950X",
        "gpu_name": "GeForce RTX 4090",
        "ram_gb": 100.0,  
        "storage_gb": 4000.0,
        "os_ver": 19.0
    },
    preferences={
        "genres": ["Strategy"],
        "modes": ["Single-player"]
    }
)

# 4. Marco: Perfect Compatibility with Cyberpunk 2077\n",
user_marco= create_user_with_existing_hardware(
    user_name="Marco",
    age=30,
    budget=20.0,
    pc_specs={
        "cpu_name": "Core i7-6700",
        "gpu_name": "Arc A380",
        "ram_gb": 12.0,
        "storage_gb": 500.0,
        "os_ver": 10.0
    },
    preferences={
        "genres": ["Simulation", "Indie"],
        "modes": ["Single-player"]
    }
)

print("\nUsers created successfully.")




--- Creating Test Users ---
[MATCH] CPU found: CPU_Intel_Core_i7_4790K
[MATCH] GPU found: GPU_NVIDIA_GeForce_GTX_1050
[MATCH] OS found: OS_Windows_11
[MATCH] Standard RAM found: RAM_64GB
[MATCH] CPU found: CPU_Intel_Core_i5_6600K
[MATCH] GPU found: GPU_NVIDIA_GeForce_GTX_1060
[MATCH] OS found: OS_Windows_10
[MATCH] Standard RAM found: RAM_8GB
[WARN] CPU 'AMD Ryzen 9 7950X' not found. Creating local instance.
[WARN] GPU 'GeForce RTX 4090' not found. Creating local instance.
[WARN] OS 'Windows 19.0' not found. Creating local instance.
[INFO] Standard RAM '100GB' not found. Creating new STANDARD individual.


In [7]:
# --- USERS FROM ATTACHED FILES (1-5) ---
#second attempt
# 4. Lucas: The Value-For-Money Gamer (Hardware from files)
user_lucas = create_user_with_existing_hardware(
    user_name="Lucas",
    age=22,
    budget=60.0,
    pc_specs={
        "cpu_name": "AMD Ryzen 5 3600",       # From cpu_filtered.json
        "gpu_name": "RTX 2060",               # From gpu_filtered.json
        "ram_gb": 16.0,
        "storage_gb": 1000.0,
        "os_ver": 10.0
    },
    preferences={
        "genres": ["Action", "RPG", "FPS"],
        "modes": ["Multiplayer", "Co-op"]
    }
)

# 5. Sarah: Mid-Range Casual (Hardware from files)
user_sarah = create_user_with_existing_hardware(
    user_name="Sarah",
    age=28,
    budget=45.0,
    pc_specs={
        "cpu_name": "Intel Core i5-10400",    # From cpu_filtered.json
        "gpu_name": "NVIDIA GeForce GTX 1660",# From gpu_filtered.json
        "ram_gb": 16.0,
        "storage_gb": 512.0,
        "os_ver": 11.0
    },
    preferences={
        "genres": ["Adventure", "Indie", "Puzzle"],
        "modes": ["Single-player"]
    }
)

# 6. Ben: Older Enthusiast Build (Hardware from files)
user_ben = create_user_with_existing_hardware(
    user_name="Ben",
    age=34,
    budget=100.0,
    pc_specs={
        "cpu_name": "Intel Core i7-7700K",    # From cpu_filtered.json
        "gpu_name": "NVIDIA GeForce GTX 1060",# From gpu_filtered.json
        "ram_gb": 32.0,
        "storage_gb": 2000.0,
        "os_ver": 10.0
    },
    preferences={
        "genres": ["Strategy", "Simulation"],
        "modes": ["Single-player", "Multiplayer"]
    }
)

# 7. Elena: Budget eSports Player (Hardware from files)
user_elena = create_user_with_existing_hardware(
    user_name="Elena",
    age=19,
    budget=25.0,
    pc_specs={
        "cpu_name": "AMD Ryzen 5 2600",       # From cpu_filtered.json
        "gpu_name": "AMD Radeon RX 580",      # From gpu_filtered.json
        "ram_gb": 16.0,
        "storage_gb": 256.0,
        "os_ver": 10.0
    },
    preferences={
        "genres": ["Shooter", "MOBA"],
        "modes": ["Multiplayer"]
    }
)

# 8. Tom: Entry-Level / Office PC (Hardware from files)
user_tom = create_user_with_existing_hardware(
    user_name="Tom",
    age=45,
    budget=15.0,
    pc_specs={
        "cpu_name": "Intel Core i3-12100",        # From cpu_filtered.json
        "gpu_name": "NVIDIA GeForce GTX 1050 Ti", # From gpu_filtered.json
        "ram_gb": 8.0,
        "storage_gb": 500.0,
        "os_ver": 11.0
    },
    preferences={
        "genres": ["Sports", "Racing"],
        "modes": ["Single-player", "Local Co-op"]
    }
)

# --- USERS WITH MODERN / WILDCARD HARDWARE (6-10) ---

# 9. Chloe: The Professional Streamer (Modern High-End)
user_chloe = create_user_with_existing_hardware(
    user_name="Chloe",
    age=24,
    budget=300.0,
    pc_specs={
        "cpu_name": "Intel Core i9-13900K",
        "gpu_name": "NVIDIA GeForce RTX 4080",
        "ram_gb": 64.0,
        "storage_gb": 4000.0,
        "os_ver": 11.0
    },
    preferences={
        "genres": ["Battle Royale", "Horror", "Action"],
        "modes": ["Multiplayer", "Co-op"]
    }
)

# 10. Max: The Gaming Laptop User
user_max = create_user_with_existing_hardware(
    user_name="Max",
    age=21,
    budget=50.0,
    pc_specs={
        "cpu_name": "Intel Core i7-11800H",
        "gpu_name": "NVIDIA GeForce RTX 3060 Laptop GPU",
        "ram_gb": 16.0,
        "storage_gb": 1000.0,
        "os_ver": 11.0
    },
    preferences={
        "genres": ["RPG", "MMORPG"],
        "modes": ["Multiplayer"]
    }
)

# 11. Sophie: Modern Mid-Range Student
user_sophie = create_user_with_existing_hardware(
    user_name="Sophie",
    age=20,
    budget=40.0,
    pc_specs={
        "cpu_name": "AMD Ryzen 5 5600X",
        "gpu_name": "AMD Radeon RX 6600",
        "ram_gb": 16.0,
        "storage_gb": 1000.0,
        "os_ver": 10.0
    },
    preferences={
        "genres": ["Simulation", "Cozy", "Adventure"],
        "modes": ["Single-player"]
    }
)

# 12. Retro_Dave: The Legacy Gamer (Older OS/Hardware)
user_retro_dave = create_user_with_existing_hardware(
    user_name="Retro_Dave",
    age=38,
    budget=20.0,
    pc_specs={
        "cpu_name": "Intel Core i5-2500K",
        "gpu_name": "NVIDIA GeForce GTX 970",
        "ram_gb": 8.0,
        "storage_gb": 250.0,
        "os_ver": 7.0
    },
    preferences={
        "genres": ["RTS", "Classic RPG"],
        "modes": ["Single-player"]
    }
)

# 13. Alex_Pro: Ultra-Enthusiast / Sim Rig
user_alex_pro = create_user_with_existing_hardware(
    user_name="Alex_Pro",
    age=29,
    budget=600.0,
    pc_specs={
        "cpu_name": "AMD Ryzen 7 7800X3D",
        "gpu_name": "NVIDIA GeForce RTX 4090",
        "ram_gb": 128.0,
        "storage_gb": 8000.0,
        "os_ver": 11.0
    },
    preferences={
        "genres": ["Flight Simulation", "Racing"],
        "modes": ["Single-player", "VR"]
    }
)

[MATCH] CPU found: CPU_AMD_Ryzen_5_3600
[MATCH] GPU found: GPU_RTX_2060
[MATCH] OS found: OS_Windows_10
[MATCH] Standard RAM found: RAM_16GB
[MATCH] CPU found: CPU_Intel_Core_i5_10400
[MATCH] GPU found: GPU_NVIDIA_GeForce_GTX_1660
[MATCH] OS found: OS_Windows_11
[MATCH] Standard RAM found: RAM_16GB
[WARN] CPU 'Intel Core i7-7700K' not found. Creating local instance.
[MATCH] GPU found: GPU_NVIDIA_GeForce_GTX_1060
[MATCH] OS found: OS_Windows_10
[MATCH] Standard RAM found: RAM_32GB
[MATCH] CPU found: CPU_AMD_Ryzen_5_2600
[MATCH] GPU found: GPU_AMD_Radeon_RX_580
[MATCH] OS found: OS_Windows_10
[MATCH] Standard RAM found: RAM_16GB
[MATCH] CPU found: CPU_Intel_Core_i3_12100
[MATCH] GPU found: GPU_NVIDIA_GeForce_GTX_1050_Ti
[MATCH] OS found: OS_Windows_11
[MATCH] Standard RAM found: RAM_8GB
[WARN] CPU 'Intel Core i9-13900K' not found. Creating local instance.
[WARN] GPU 'NVIDIA GeForce RTX 4080' not found. Creating local instance.
[MATCH] OS found: OS_Windows_11
[MATCH] Standard RAM found: R

In [ ]:
# Rules for the ontology

with onto:
    # Properties for "Perfect Match"
    if "isPerfectMatchFor" not in locals():
        class isPerfectMatchFor(ObjectProperty):
            domain = [VideoGame]
            range = [User]
            label = ["is a perfect match for"]

    ## Define the "Perfect Compatibility Award" easter egg
    if "isPerfectHardwareMatch" not in locals():
        class isPerfectHardwareMatch(ObjectProperty):
            domain = [MinimumRequirement]
            range = [Computer]
            label = ["Compatibility Award"]

    # Rules
    # Rule 1: Suggested Games (based on user preferences)
    rule_perfect = Imp()
    rule_perfect.set_as_rule(
        """VideoGame(?v), User(?u), 
           hasGenre(?v, ?g), preferredGenre(?u, ?g),
           hasPlayerMode(?v, ?m), preferredPlayerMode(?u, ?m) 
           -> isPerfectMatchFor(?v, ?u)"""
    )

    # Rule 2: Compatibility Award (Exact match between requirements and hardware)
    rule_perfect_hw = Imp()
    rule_perfect_hw.set_as_rule(
        """MinimumRequirement(?req), Computer(?c), 
           hasCPU(?c, ?u_cpu), hasBenchmarkScore(?u_cpu, ?score_cpu),
           hasGPU(?c, ?u_gpu), hasBenchmarkScore(?u_gpu, ?score_gpu),
           hasOS(?c, ?u_os), hasOSVersionValue(?u_os, ?os_ver),
           requiresCPUBenchmarkScore(?req, ?score_cpu), 
           requiresGPUBenchmarkScore(?req, ?score_gpu), 
           requiresMinOSVersionValue(?req, ?os_ver)
           -> isPerfectHardwareMatch(?req, ?c)""",
        namespaces=[onto]
    )

In [8]:

#  SAVE ONTOLOGY
output_file = "gameMatcher_ABox.owl"
# Save the ontology to a local file
onto.save(file=output_file)
print("ABox created successfully!")
print(f"Ontology saved to: {output_file}")

ABox created successfully!
Ontology saved to: gameMatcher_ABox.owl
